#### Adapted from https://github.com/touldimos/WH

In [18]:
!pip install numpy fluids pint CoolProp pandas

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, interactive, fixed, interact_manual
import fluids,pint
import CoolProp as cp
ureg = pint.UnitRegistry()#(fmt_locale='es_ES')

#### Onda acústica a través de una cañería

 $$a^2 = \frac{K_f}{\rho} \frac{1}{1 + \left(K_f/E   (1 - \nu_p^2)  (D/ e)\right)}$$
Reconociendo $a$ como la velocidad de la onda acústica a partir de 
* $K_f$ módulo elástico del fluido.
* $\rho$ densidad del fluido.
* Diámetro de la cañería
* E módulo elástico del material del caño.
* \nu_p coeficiente de Poisson del material del caño.
* $e$ espesor del caño.


In [14]:
# Geometría
D = 0.77  #diámetro [m]
e = 0.01  #espesor [m]
l = 500  #longitud [m]

#Propiedades de fluidos  / cañería
                     
K = fluids.roughness_Farshad('Carbon steel, honed bare', D)*10   #Rugosidad de la cañería [m]
eD = K / D

rho = cp.CoolProp.PropsSI('D', 'T', 25 + 273, 'P', 101.3e3, 'Water')  #densidad [kg/m**3]
mu = cp.CoolProp.PropsSI('V', 'T', 25 + 273, 'P', 101.3e3,
                         'Water')  #viscosidad [kg  / m s]
nu = mu / rho #Viscosidad Cinemática [m**2/s]

g = 9.81  #Gravedad [m/s**2]
E = 78 * 1e9  #Elasticidad de la cañería,acero[Pa]
#Kf = 1.96*10**9                     #Elasticidad del fluido [Pa] = 1/ iso_compressibility [1/Pa]
Isothermal_compressibility = cp.CoolProp.PropsSI('isothermal_compressibility',
                                                 'T', 25 + 273, 'P', 101.3e3,
                                                 'Water')
Kf = 1 / Isothermal_compressibility

Poisson = 0.3  #Coeficiente de Poisson, acero

#Condiciones Iniciales
elevacion_salida = 10  #altura salida [m]
elevacion_entrada = 30 #altura entrada [m]
Q = 1.988   #caudal [m**3/s]

hfl = 10  #Pérdidas localizadas  [% de delta perdidas friccion]

#Condiciones de cálculo

nodes = 51    # numero de nodeos
tclose = .1   #tiempo de cierre de la válvula
hzero = 120   #
tmax = 50     #maximo tiempo de cálculo

Din = D - 2 * e   #diámetro interno [m]
A = (np.pi * Din**2) * 0.25   #Área de la cañería [m**2]
v = Q / A                     #Velocidad de Flujo  [m/s]
 
Re = Din * v / nu             #Número de Reynolds

f = fluids.friction_factor(Re, K) #factor de fricción
hf = f * l * v**2 / (Din * 2 * g) #caída por fricción en altura [m]
hl = hfl / 100 * hf               #caída por pérdidas localizadas [m]
dh = hf + hl                      #caídas totales [m]

a = np.sqrt(Kf / rho) / (np.sqrt(1 + (Kf * D * (1 - Poisson**2)) / (E * e))) #velocidad sonido
#dp = a * v / g                                   # 


dx = l / (nodes - 1)  #delta x cálculos
dt = dx / a           #delta t cálculos por relación de características
delta_elevacion = (elevacion_entrada - elevacion_salida) / (nodes - 1) #delta de elevacion
ca = (g * np.pi * D**2) / (4 * a)      #constante para cálculo de características ~C_1
step = np.arange(0, tmax + dt, dt)     #vector pasos de tiempo

In [15]:
#Cálculo
#inicializa vectores salida
x = np.zeros(nodes)
h = np.zeros(nodes)
hlow = np.zeros(nodes)
hhigh = np.zeros(nodes)
pipez = np.zeros(nodes)
head = np.zeros(nodes)

for i in range(nodes):
    x[i] = i * dx
    h[i] = hzero - dh * i / nodes
    hlow[i] = h[i]
    hhigh[i] = h[i]
    pipez[i] = elevacion_entrada - delta_elevacion * i
    head[i] = h[i] - pipez[i]

Qic = np.ones(nodes) * Q
Vic = np.ones(nodes) * v
Hic = h

#funcion del método de características
def chars(nodes, Q, D, n, K, H, ca, dt, hzero, V0, t, tclose, A):

    cm_ = np.zeros(nodes)
    cn_ = np.zeros(nodes)
    Hnew = np.zeros(nodes)
    Qnew = np.zeros(nodes)
    Vnew = np.zeros(nodes)
    for i in range(1, nodes):
        if Q[i - 1] != 0:
            vi = Q[i - 1] / A
            Re = Din * vi / n
            f = fluids.friction_factor(Re, K)

        else:
            f = 0
        cfa = f * dt / (np.pi * D**3 / 2)
        cm = Q[i - 1] + ca * H[i - 1] - cfa * Q[i - 1] * abs(Q[i - 1])
        cm_[i] = cm
    for i in range(0, nodes - 1):
        if Q[i + 1] != 0:
            vi = Q[i + 1] / A
            Re = Din * vi / n
            f = fluids.friction_factor(Re, K)
        else:
            f = 0
        cfb = f * dt / (np.pi * D**3 / 2)
        cn = Q[i + 1] - ca * H[i + 1] - cfb * Q[i + 1] * abs(Q[i + 1])
        cn_[i] = cn
    Hnew = 0.5 * (cm_ - cn_) / ca
    Hnew[0] = hzero
    Qnew = cn_ + ca * Hnew
    Vnew = 4 * Qnew / (np.pi * D**2)
    Hnew = 0.5 * (cm_ - cn_) / ca
    Hnew[0] = hzero
    #ley de Cierre de la Válvula
    if t > tclose:
        Vnew[-1] = 0
    else:
        Vnew[-1] = V0 - (t / tclose * V0)
    Qnew[-1] = Vnew[-1] * np.pi * D**2 / 4
    Hnew[-1] = (cm_[-1] - Qnew[-1]) / ca
    return Hnew, Qnew, Vnew


H = []
Q = []
V = []

H.append(Hic)
Q.append(Qic)
V.append(Vic)

H.append(
    chars(nodes, Qic, Din, nu, K, Hic, ca, dt, hzero, v, dt, tclose, A)[0])
Q.append(
    chars(nodes, Qic, Din, nu, K, Hic, ca, dt, hzero, v, dt, tclose, A)[1])
V.append(
    chars(nodes, Qic, Din, nu, K, Hic, ca, dt, hzero, v, dt, tclose, A)[2])

for i in range(1, len(step)):
    H.append(
        chars(nodes, Q[i], Din, nu, K, H[i], ca, dt, hzero, v, step[i], tclose,
              A)[0])
    Q.append(
        chars(nodes, Q[i], Din, nu, K, H[i], ca, dt, hzero, v, step[i], tclose,
              A)[1])
    V.append(
        chars(nodes, Q[i], Din, nu, K, H[i], ca, dt, hzero, v, step[i], tclose,
              A)[2])

Head = pd.DataFrame(H)
Discharge = pd.DataFrame(Q)
Velocity = pd.DataFrame(V)

Head.head(10)

,0,1,2,3,4,5,6,7,8,9,...,41,42,43,44,45,46,47,48,49,50
0,120.0,119.809107,119.618214,119.427321,119.236428,119.045536,118.854643,118.66375,118.472857,118.281964,...,112.173392,111.982499,111.791606,111.600713,111.409820,111.218927,111.028035,110.837142,110.646249,110.455356
1,120.0,119.809107,119.618214,119.427321,119.236428,119.045536,118.854643,118.66375,118.472857,118.281964,...,112.173392,111.982499,111.791606,111.600713,111.409820,111.218927,111.028035,110.837142,110.646249,153.997028
2,120.0,119.809107,119.618214,119.427321,119.236428,119.045536,118.854643,118.66375,118.472857,118.281964,...,112.173392,111.982499,111.791606,111.600713,111.409820,111.218927,111.028035,110.837142,154.169800,154.019966
3,120.0,119.809107,119.618214,119.427321,119.236428,119.045536,118.854643,118.66375,118.472857,118.281964,...,112.173392,111.982499,111.791606,111.600713,111.409820,111.218927,111.028035,154.342579,154.192729,197.597826
4,120.0,119.809107,119.618214,119.427321,119.236428,119.045536,118.854643,118.66375,118.472857,118.281964,...,112.173392,111.982499,111.791606,111.600713,111.409820,111.218927,154.515363,154.365497,197.754655,241.139458
5,120.0,119.809107,119.618214,119.427321,119.236428,119.045536,118.854643,118.66375,118.472857,118.281964,...,112.173392,111.982499,111.791606,111.600713,111.409820,154.688154,154.538272,197.911492,241.282550,284.749115
6,120.0,119.809107,119.618214,119.427321,119.236428,119.045536,118.854643,118.66375,118.472857,118.281964,...,112.173392,111.982499,111.791606,111.600713,154.860950,154.711052,198.068339,241.425652,284.880655,328.318176
7,120.0,119.809107,119.618214,119.427321,119.236428,119.045536,118.854643,118.66375,118.472857,118.281964,...,112.173392,111.982499,111.791606,155.033753,154.883838,198.225195,241.568765,285.012204,328.440365,371.950862
8,120.0,119.809107,119.618214,119.427321,119.236428,119.045536,118.854643,118.66375,118.472857,118.281964,...,112.173392,111.982499,155.206561,155.056631,198.382061,241.711889,285.143763,328.562561,372.065889,415.538584
9,120.0,119.809107,119.618214,119.427321,119.236428,119.045536,118.854643,118.66375,118.472857,118.281964,...,112.173392,155.379375,155.229429,198.538935,241.855023,285.275331,328.684764,372.180920,415.648660,459.185528


In [16]:
time = Head.index * dt
ref_tmin = -10
def plot_hammer(i = 0):
    plt.figure(figsize = (15, 5))
    plt.plot(time, Head[i], 'b', zorder = 11)
    plt.vlines(tclose, -abs(Head[10].min()), Head[10].max(), 'r', ls = '--', alpha = 0.5, zorder = 10)
    plt.hlines(pipez[i], 0, time[-1], 'darkgreen', alpha = 0.5, zorder = 8)
    plt.hlines(0, ref_tmin, time[-1], 'black', zorder = 9)
    plt.vlines(ref_tmin, -abs(Head[10].min()), Head[10].max(), 'black', zorder = 8)
    plt.legend(['Onda presión', 'Tiempo de cierre {} segs'.format(tclose), 'Nivel inicial'],loc='best')
    plt.title('Node {}'.format(i))
    plt.xlabel('Tiempo [seg]')
    plt.ylabel('Altura [m]')
    plt.grid();

interact(plot_hammer, i = (0, (nodes - 1), 1));

interactive(children=(IntSlider(value=0, description='i', max=50), Output()), _dom_classes=('widget-interact',…

In [17]:
def plot_hammer_line(i = 0):
    plt.figure(figsize = (15, 5))
    plt.plot(Head.loc[i], 'b', zorder = 10)
    plt.plot(pipez, 'darkgreen', alpha = 0.8, zorder = 10)
    plt.plot(Head.max(axis = 0), 'r--')
    plt.plot(Head.min(axis = 0), 'y--')
    plt.hlines(0, 0, nodes - 1, 'black', zorder = 9)
    plt.vlines(0, -abs(Head[10].min()), Head[10].max(), 'black', zorder = 8)
    plt.legend(['Onda de presión', 'Nivel inicial', 'Máximo', 'Mínimo'])
            
    plt.title('Tiempo {:.3f} seg'.format(i * dt))
    plt.xlabel('Nodos [-]')
    plt.ylabel('Altura [m]')
    plt.grid();
    
interact(plot_hammer_line, i = (0, len(step), 10));

interactive(children=(IntSlider(value=0, description='i', max=4310, step=10), Output()), _dom_classes=('widget…